<a href="https://colab.research.google.com/github/Bishop1303/ML_PySpark/blob/dev/ML_Regularization_flightmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Carica il drive con i dati:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Getting the softwares:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark


# To use spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

# SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark



# Flight duration model: More features!

Add more features to model will not necessarily result in a better model. Adding some features might improve the model. Adding other features might make it worse.

More features will always make the model more complicated and difficult to interpret.

These are the features include in the model:

* **km**
* **org** (origin airport, one-hot encoded, 8 levels)
* **depart** (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)
* **dow** (departure day of week, one-hot encoded, 7 levels) and
* **mon** (departure month, one-hot encoded, 12 levels).  

These have to been assembled into the features column, which is a *sparse representation* of 32 columns (*one-hot encoding* produces a number of columns which is one fewer than the number of levels).

In [9]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import round


### DATA PREPARATION ###

# Read raw data to pyspark
flights = spark.read.csv("/content/drive/My Drive/flights.csv", inferSchema=True, header=True, mode='FAILFAST')
#flights.show()
#flights.printSchema()

### DATA PREPARATION ###

# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Remove records with missing 'delay' values or NA values
flights_none_missing = flights_drop_column.filter((flights_drop_column.delay.isNotNull()) & \
                                                  (flights_drop_column.delay != 'NA'))

# Test delay
flights_none_missing = flights_none_missing.withColumn('delay', flights_none_missing['delay'].cast('int'))

# Check on dataframe
#print('The Schema is: ')
#flights_none_missing.printSchema()
#print('=====================================================')
#print('Informative rows after dropping malformed: ',flights_none_missing.count())


# Conversion: 'mile' to 'km' and drop 'mile' column
flights_km = flights_none_missing.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')

# Creating 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check records
#flights_km.show(5)
#flights_km.printSchema()

idx_input_cols=['carrier', 'org']
idx_output_cols=['carrier_idx', 'org_idx']

# Create an indexer
indexer = StringIndexer(inputCols=idx_input_cols, outputCols=idx_output_cols)

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# Check result
#flights_indexed.show(5,False)

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights_indexed data
onehot = onehot.fit(flights_indexed)
flights_onehot = onehot.transform(flights_indexed)

# Check the results
#flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

# Create buckets at 3 hour intervals through the day
splits=[0,3,6,9,12,15,18,21,24]
buckets = Bucketizer(splits=splits, inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights_onehot)
#bucketed.select('depart','depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
#flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

# Create an assembler object
assembler = VectorAssembler(inputCols=['km','org_dummy','depart_dummy','dow','mon'], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_onehot)

# Check the resulting column
flights_assembled = flights_assembled.select('mon','dom','dow','carrier','org','depart','duration','delay','km',
                                             'org_idx','org_dummy','depart_bucket','depart_dummy','features')

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=7)


In [10]:
### MODEL EVALUATION ###

# Fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Make predictions on testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print('Regression coefficients are:\n ',coeffs)

The test RMSE is 10.5733817087635
Regression coefficients are:
  [0.07440217199394049,27.696657449729237,20.590145822897725,51.83452291203616,45.99926965887334,15.437053513762223,17.931565316893604,17.79852553066936,-14.805760419640027,1.5200493085884204,3.9554969638546966,6.913548613724635,4.669555628501808,8.729855197467081,8.566405052255341,-0.03641255730024152,0.08740640199575375]


#Flight duration model: Regularisation!

The model performed well on testing data, but with so many coefficients it was difficult to interpret.

Using Lasso regression (regularized with a L1 penalty) to create a more parsimonious model.  
Many of the coefficients in the resulting model will be set to zero. This means that only a subset of the predictors actually contribute to the model. The objective is to produce the simplier possible model that explain well the data.  

A specific value for the regularization strength will be used ($\alpha =1$). Later a better value will be calculated using cross validation.


In [ ]:
# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)